In [2]:
import numpy as np
import pickle
#from build_database import flux_obj
from scipy import interpolate
import matplotlib.pyplot as plt
#from matplotlib import pyplot as plt
from GLD_file_tools import GLD_file_tools
from satellite import Satellite
import datetime
import ephem
from coordinate_structure import coordinate_structure
from coordinate_structure import transform_coords
from longitude_scaling import longitude_scaling
from ionoAbsorp import ionoAbsorp
import os
#from mpl_toolkits.basemap import Basemap
from precip_model import precip_model
import itertools
from measurement_model import measurement_model
import random
#from scaling import get_time_scaling, get_map_scaling

In [1]:
with open('/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/filled_measurements.pkl','rb') as file:
    d = pickle.load(file)

print "loaded %g entries" % len(d.keys())

loaded 116250 entries


In [2]:
# Generator objects
GLD_root = '/Users/austin/FUSE/shared/users/asousa/WIPP/fluxMDP/GLD'

# sat_TLE  = ["1 40378U 15003C   15293.75287141  .00010129  00000-0  48835-3 0  9990",
#             "2 40378  99.1043 350.5299 0153633 201.4233 158.0516 15.09095095 39471"]

# --------------- UPDATED TLE, 12.3.2015 -----------------------------------------
sat_TLE = ["1 40378U 15003C   15337.63361572  .00004913  00000-0  23452-3 0  9999",
           "2 40378  99.0977  42.5740 0152371  57.8830 303.7080 15.09952916 46106"]
# --------------- UPDATED TLE, 12.3.2015 -----------------------------------------

# Satellite object:
sat = Satellite(sat_TLE[0], sat_TLE[1],'Firebird 4')

# Measurement object:
f = measurement_model(database = 'database_dicts.pkl', GLD_root=GLD_root, multiple_bands = True)


bands = dict()
bands['low'] = [1,2]
bands['mid'] = [3, 4, 5]
bands['high']= [6, 7, 8]


In [9]:
# ---- Calculate the stuff we didn't do yet!
d = dict()

cur_time = datetime.datetime(2015,12,3,0,0,0)
stoptime = datetime.datetime(2015,12,5,0,0,0)
td = datetime.timedelta(seconds=30)

while cur_time < stoptime:
    d[cur_time] = dict()
    cur_time += td
    

In [29]:
keylist = sorted(d.keys())

all_options = ['off','continuous']
for k in keylist[21:30]:
    #print d[k].keys()
    sat.compute(k)
    sat.coords.transform_to('geomagnetic')

    todo = set(all_options) - set(d[k].keys())
    print "timestep: " + str(k) + ", doing: " + str(todo)
    for act in todo:
        if act=='off':
            meas = 0
        if act=='continuous':
            meas = f.get_measurement(k,sat.coords, mode='continuous')
        if act in ['low','mid','high']:
            meas = f.get_measurement(k, sat.coords, mode='banded', bands=bands[act])

        d[k][act] = meas

timestep: 2015-10-08 18:40:30, doing: set(['off'])
timestep: 2015-10-08 18:41:00, doing: set(['off'])
timestep: 2015-10-08 18:41:30, doing: set(['off'])
timestep: 2015-10-08 18:42:00, doing: set(['continuous'])
timestep: 2015-10-08 18:42:30, doing: set([])
timestep: 2015-10-08 18:43:00, doing: set(['continuous'])
timestep: 2015-10-08 18:43:30, doing: set(['off'])
timestep: 2015-10-08 18:44:00, doing: set(['continuous'])
timestep: 2015-10-08 18:44:30, doing: set(['continuous'])


In [10]:
# Find un-calculated measurements:
keylist = sorted(d.keys())
all_options = ['off','continuous','low','mid','high']

out_dir = '/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/'

todo_list = []
for k in keylist:
    cur_list = set(all_options) - set(d[k].keys())
    for task in cur_list:
        todo_list.append([k, task])
print np.shape(todo_list)


total_tasks = np.shape(todo_list)[0]

i = 0
blocksize = 500
total_blocks = 0
while i < total_tasks:
    tmp = todo_list[i:i+blocksize]
    i += blocksize
    total_blocks += 1
    print np.shape(tmp)
    
    with open(os.path.join(out_dir, "block_%g.pkl"% total_blocks),'wb') as file:
        pickle.dump(tmp,file)
print total_blocks

(28800, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(500, 2)
(300, 2)
58


In [11]:
with open('/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/block_58.pkl','rb') as file:
    tmp = pickle.load(file)

for row in tmp:
    print row

[datetime.datetime(2015, 12, 4, 23, 30), 'high']
[datetime.datetime(2015, 12, 4, 23, 30), 'continuous']
[datetime.datetime(2015, 12, 4, 23, 30), 'off']
[datetime.datetime(2015, 12, 4, 23, 30), 'low']
[datetime.datetime(2015, 12, 4, 23, 30), 'mid']
[datetime.datetime(2015, 12, 4, 23, 30, 30), 'high']
[datetime.datetime(2015, 12, 4, 23, 30, 30), 'continuous']
[datetime.datetime(2015, 12, 4, 23, 30, 30), 'off']
[datetime.datetime(2015, 12, 4, 23, 30, 30), 'low']
[datetime.datetime(2015, 12, 4, 23, 30, 30), 'mid']
[datetime.datetime(2015, 12, 4, 23, 31), 'high']
[datetime.datetime(2015, 12, 4, 23, 31), 'continuous']
[datetime.datetime(2015, 12, 4, 23, 31), 'off']
[datetime.datetime(2015, 12, 4, 23, 31), 'low']
[datetime.datetime(2015, 12, 4, 23, 31), 'mid']
[datetime.datetime(2015, 12, 4, 23, 31, 30), 'high']
[datetime.datetime(2015, 12, 4, 23, 31, 30), 'continuous']
[datetime.datetime(2015, 12, 4, 23, 31, 30), 'off']
[datetime.datetime(2015, 12, 4, 23, 31, 30), 'low']
[datetime.datetime(2

In [4]:
# Load database, add calculated stuff to it, save it out:
# with open('/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/filled_measurements.pkl','rb') as file:
#     d = pickle.load(file)
d = dict()
print "loaded %g entries" % len(d.keys())

# d is existing dictionary: d[timestamp] = dict[measurement-type]
for root, dirs, files in os.walk('/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/'):
    for filename in files:
        if 'completed_' in filename:
            print os.path.join(root,filename)
            with open(os.path.join(root,filename),'rb') as f:
                grid = pickle.load(f)
                print np.shape(grid)

                for row in grid:
                    #print row
                    if not row[0] in d.keys():
                        d[row[0]] = dict()
                    
                    d[row[0]][row[1]] = row[2]

                    
with open('/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/measurements_for_alex.pkl','wb') as file:
    pickle.dump(d,file)

loaded 0 entries
/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/completed_block_38.pkl
(500, 3)
/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/completed_block_10.pkl
(500, 3)
/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/completed_block_27.pkl
(500, 3)
/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/completed_block_19.pkl
(500, 3)
/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/completed_block_30.pkl
(500, 3)
/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/completed_block_47.pkl
(500, 3)
/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/completed_block_50.pkl
(500, 3)
/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/completed_block_52.pkl
(500, 3)
/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_alex/completed_block_58.pkl
(300, 3)
/Users/austin/FUSE2/shared/users/asousa/WIPP/fluxMDP/outputs/for_

In [10]:

print "starting"
for k in d.keys():
    
    if not len(d[k].keys())==5:
        print "mismatch at ", k
        print d[k].keys()
print "done"

print np.min(d.keys())
print np.max(d.keys())

starting
mismatch at  2015-11-17 22:49:30
['continuous', 'off', 'low']
mismatch at  2015-11-17 23:01:30
['high', 'continuous', 'off']
mismatch at  2015-11-17 22:51:30
['continuous', 'off', 'mid']
mismatch at  2015-11-17 23:15:30
['continuous', 'off', 'mid']
mismatch at  2015-11-17 23:13:30
['high', 'continuous', 'off']
mismatch at  2015-11-17 23:11:30
['continuous', 'off', 'low']
mismatch at  2015-11-17 23:22:30
['continuous', 'off']
mismatch at  2015-11-17 23:09:30
['high', 'continuous', 'off', 'mid']
mismatch at  2015-11-17 23:20:30
['high', 'continuous', 'off', 'mid']
mismatch at  2015-11-17 22:56:30
['high', 'continuous', 'off']
mismatch at  2015-11-18 00:07:30
['continuous', 'off', 'mid']
mismatch at  2015-11-17 23:18:30
['high', 'continuous', 'off', 'low']
mismatch at  2015-11-17 22:58:30
['continuous', 'off', 'mid']
mismatch at  2015-11-17 23:16:30
['continuous', 'off']
mismatch at  2015-11-18 00:32:30
['high', 'continuous', 'off', 'mid']
mismatch at  2015-11-17 23:30:30
['conti